In [2]:
!pip install azure-storage-blob


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 16.8 MB/s eta 0:00:00


In [9]:
# Connecting Azure Blob
from azure.storage.blob import BlobServiceClient
from io import StringIO
import pandas as pd

#real connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=diabetes;AccountKey=w1HVHfh4k07Y0HVrkBy/trNDJSys6vZOEpHEQhA2gd/dODl09ASk4/5ylTeI0noRAtJxzOWbKUJb+AStSMgunA==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)




In [10]:
# 📂 Step 4: Set your container name
container_name = 'datasets'  # Replace if your container has a different name

# Optional sanity check: List all files in the container
container_client = blob_service_client.get_container_client(container=container_name)

print("📁 Files inside container:")
for blob in container_client.list_blobs():
    print("➡️", blob.name)


📁 Files inside container:
➡️ Diabetes Classification.csv


In [11]:
# 📄 Step 5: Access and read the blob
blob_name = 'Diabetes Classification.csv'  # Use exact name from list above

blob_client = blob_service_client.get_blob_client(container='datasets', blob='Diabetes Classification.csv')

# Download blob content as text
blob_data = blob_client.download_blob().content_as_text()

# Read CSV from text into DataFrame
df = pd.read_csv(StringIO(blob_data))

# ✅ Preview your data
df.head()



,Unnamed: 0,Age,Gender,BMI,Chol,TG,HDL,LDL,Cr,BUN,Diagnosis
0,0,50,F,24,4.2,0.9,2.4,1.4,46.0,4.7,0
1,1,26,M,23,3.7,1.4,1.1,2.1,62.0,4.5,0
2,2,33,M,21,4.9,1.0,0.8,2.0,46.0,7.1,0
3,3,45,F,21,2.9,1.0,1.0,1.5,24.0,2.3,0
4,4,50,F,24,3.6,1.3,0.9,2.1,50.0,2.0,0


In [12]:
#1: Drop Unnecessary Columns (like index column)
df.drop(columns=['Unnamed: 0'], inplace=True)

# 2: Encode Categorical Variables (e.g., Gender)
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})
print("Gender unique values:", df['Gender'].unique())


# ✅ Step 3: Check for Null Values
print("Missing values:\n", df.isnull().sum())

# ✅ Step 4: Split into Features and Target
X = df.drop('Diagnosis', axis=1)
y = df['Diagnosis']


Gender unique values: [ 0.  1. nan]
Missing values:
 Age          0
Gender       1
BMI          0
Chol         0
TG           0
HDL          0
LDL          0
Cr           0
BUN          0
Diagnosis    0
dtype: int64


In [14]:
# ✅ Step 5: Train-Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Step 6: Feature Scaling (Standardization)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [15]:
# ✅ Step 7: Convert to DataFrames (for easy viewing)
import pandas as pd

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

# Preview the scaled data
X_train_scaled_df.head()


,Age,Gender,BMI,Chol,TG,HDL,LDL,Cr,BUN
0,-0.926521,-1.315499,-1.077445,0.000425,-0.689227,-0.006411,0.066486,-0.575180,-0.713096
1,-0.426895,-1.315499,-0.609956,-0.791865,-0.823399,0.149582,-0.935781,-0.735412,-0.180640
2,-1.711648,0.760168,-1.077445,-0.270357,-0.532693,-0.250150,-0.302770,0.289404,-0.379580
3,-1.140646,-1.315499,-0.142467,-0.671517,-0.159993,-0.376895,-0.650926,-0.311465,0.234793
4,1.642985,-1.315499,-0.843700,0.461759,-0.122723,-0.376895,0.024286,-0.211320,-0.783310
